In [124]:
from jia_cidades import multi_regime, F, G, new_diagn, new_deaths, multi_regime_cidades, tot_deaths

In [125]:
from get_data import read_data, get_data, download_csv

In [126]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

sns.set_style("darkgrid")

In [204]:
metropolitana_1 = ['RIO DE JANEIRO', 'ITAGUAI', 'SEROPÉDICA', 'QUEIMADOS', 'JAPERI', 'NOVA IGUAÇU', 'MESQUITA', 'NILOPOLIS', 'BELFORD ROXO', 'SAO JOAO DO MERITI', 'DUQUE DE CAXIAS', 'MAGE']

In [203]:
arquivo_conf = 'data_municipios/dados_estado_rio/BICOVIDREAIS_22abrREV.csv'
arquivo_obito = 'data_municipios/dados_estado_rio/αbitosASCOM.csv'
df_confirmados = pd.read_csv(arquivo_conf, encoding = 'ISO-8859-1', delimiter = ';')
df_obitos = pd.read_csv(arquivo_obito, encoding = 'ISO-8859-1', delimiter = ',')
#df_confirmados = read_data(df_confirmados, 'confirmados')
#df_obitos = read_data(df_obitos, 'obitos')

In [141]:
def read_data(df, tipo):
    if tipo == 'confirmados':
        df  = df.drop(columns = ['sexo', 'idade', 'uf', 'dt_sintoma', 'evolucao', 'dt_obito', 'comorbidades'])
        #df.rename(mapper = {'FX_ETARIA' : 'Faixa Etaria', 'MUNIC_RESIDENCIA': 'Municipio', 'BAIRRO_RESIDENCIA': 'Bairro',
                               #'DT_NOT': 'Data'}, axis = 1, inplace = True)
        c = df.columns
        #df[[c[2], c[3]]] = df[[c[3], c[2]]]
        df.rename(mapper = {'dt_coleta / dt_notif':'Data'}, axis = 1, inplace = True)
        df = df.drop(labels = len(df)-1, axis = 0)
        return df
    elif tipo == 'obitos':
        df = df.drop(columns = ['SEXO', 'IDADE', 'CONFIRMAÇÃO', 'COMORBIDADE'])
        df.rename(mapper = {'FXETARIA':'Faixa Etaria', 'MUN_RES': 'Municipio', 'DT_OBITO':'Data'}, axis = 1, inplace = True)
        return df
    else:
        print('tipos são confirmados ou obitos')


def get_data(cidade, df, T_fim, T_start = '29-03-2020'):
    df_cidade = df.loc[df['municipio_res'] == cidade]
    n = len(df_cidade)
    df = df_cidade
    k = n - len(df_cidade.dropna(subset=['Data']))
    df_cidade = df_cidade.dropna(subset=['Data'])
    print('No total ' + str(k) + ' dados foram inutilizados')
    l = list(df_cidade['Data'])
    start = dt.datetime.strptime(T_start, "%d-%m-%Y")
    then = start + dt.timedelta(days=T_fim)
    days = mdates.drange(start,then,dt.timedelta(days=1))
    dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
    dados_por_dia = len(dias)*[0]
    df_cidade_new = pd.to_datetime(df_cidade['Data'], dayfirst=True, format='%d/%m/%Y')
    times =list(df_cidade_new)
    times = [str(time).split(' ')[0] for time in times]
    for i, dia in enumerate(dias):
        dados_por_dia[i] += times.count(dia)
    dados = [sum(dados_por_dia[:i]) for i in range(1, len(dados_por_dia)+1)]
    acumulado_inicial = n - dados[-1]
    dados = [acumulado_inicial + dado for dado in dados]
    return dados, dados_por_dia

In [55]:
df_conf = read_data(df_confirmados, 'confirmados')

In [56]:
df_conf

,municipio_res,Data
0,TERESOPOLIS,02/04/2020
1,RIO DE JANEIRO,08/04/2020
2,MESQUITA,30/03/2020
3,RIO DE JANEIRO,09/04/2020
4,RIO DE JANEIRO,04/04/2020
...,...,...
5546,RIO DE JANEIRO,10/04/2020
5547,NITEROI,21/04/2020
5548,RIO DE JANEIRO,12/04/2020
5549,RIO DE JANEIRO,09/04/2020


In [85]:
lst = []
for c in metropolitana_1:
    lst.append(get_data(c, df_conf, T_fim = 24)[1])

No total 230 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 2 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 1 dados foram inutilizados
No total 0 dados foram inutilizados
No total 2 dados foram inutilizados
No total 0 dados foram inutilizados
No total 9 dados foram inutilizados
No total 2 dados foram inutilizados


In [86]:
len(metropolitana_1),len(lst)

(12, 12)

In [87]:
start = dt.datetime.strptime('29-03-2020', "%d-%m-%Y")
then = start + dt.timedelta(days=24)
days = mdates.drange(start,then,dt.timedelta(days=1))
dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]

In [88]:
dic = {'DATA': dias, metropolitana_1[0]: lst[0], metropolitana_1[1]: lst[1],metropolitana_1[2]: lst[2],metropolitana_1[3]: lst[3],metropolitana_1[4]: lst[4],metropolitana_1[5]: lst[5],metropolitana_1[6]: lst[6],metropolitana_1[7]: lst[7],metropolitana_1[8]: lst[8],metropolitana_1[9]: lst[9],metropolitana_1[10]: lst[10],metropolitana_1[11]: lst[11]}

In [89]:
df_conf_estado = pd.DataFrame(dic)

In [90]:
df_conf_estado

,DATA,RIO DE JANEIRO,ITAGUAI,SEROPÉDICA,QUEIMADOS,JAPERI,NOVA IGUAÇU,MESQUITA,NILOPOLIS,BELFORD ROXO,SAO JOAO DO MERITI,DUQUE DE CAXIAS,MAGE
0,2020-03-29,44,1,0,1,0,0,0,0,1,0,4,1
1,2020-03-30,144,2,0,3,1,0,3,0,1,0,5,2
2,2020-03-31,75,0,0,3,0,0,4,2,2,0,7,2
3,2020-04-01,50,0,0,1,0,0,1,2,4,0,5,2
4,2020-04-02,104,0,0,0,2,0,3,4,4,0,6,1
5,2020-04-03,147,0,0,0,1,0,4,1,5,0,8,3
6,2020-04-04,61,0,0,1,0,0,0,0,2,0,2,0
7,2020-04-05,22,1,0,2,0,0,1,1,3,0,3,0
8,2020-04-06,192,0,0,3,1,0,11,2,9,0,16,4
9,2020-04-07,203,0,0,1,0,0,5,2,8,0,18,6


In [132]:
#df_conf_estado.to_csv('metropolitana_1_dados_estado.csv')

In [133]:
df_confirmados[df_confirmados['municipio_res'].str.contains('|'.join(metropolitana_1))]

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta / dt_notif,evolucao,dt_obito,comorbidades
1,M,88.0,RIO DE JANEIRO,RJ,07/04/2020,08/04/2020,OBITO,15/04/2020,SI
2,M,37.0,MESQUITA,RJ,29/03/2020,30/03/2020,OBITO,01/04/2020,SIM
3,M,53.0,RIO DE JANEIRO,RJ,09/04/2020,09/04/2020,OBITO,10/04/2020,SIM
4,F,85.0,RIO DE JANEIRO,RJ,26/03/2020,04/04/2020,OBITO,08/04/2020,NAO
5,M,70.0,DUQUE DE CAXIAS,RJ,30/03/2020,04/04/2020,OBITO,07/04/2020,SIM
...,...,...,...,...,...,...,...,...,...
5546,F,64.0,RIO DE JANEIRO,RJ,16/03/2020,10/04/2020,NaN,NaN,NaN
5548,M,53.0,RIO DE JANEIRO,RJ,NaN,12/04/2020,NaN,NaN,NaN
5549,F,53.0,RIO DE JANEIRO,RJ,NaN,09/04/2020,NaN,NaN,NaN
5550,F,65.0,RIO DE JANEIRO,RJ,20/03/2020,11/04/2020,NaN,NaN,NaN


In [145]:
df_obitos

,CONFIRMAÇÃO,DIVULGAÇÃO,MUNICÍPIO,SEXO,IDADE,COMORBIDADE,Unnamed: 6,Unnamed: 7
0,1.0,19/03/2020,Miguel Pereira,FEM,63,Sim,NaN,NaN
1,2.0,19/03/2020,Niterói,MAS,69,Sim,NaN,NaN
2,3.0,21/03/2020,Petrópolis,MAS,65,Sim,NaN,NaN
3,4.0,23/03/2020,Rio de Janeiro,FEM,58,Sim,NaN,NaN
4,5.0,24/03/2020,Rio de Janeiro,FEM,71,Sim,NaN,NaN
...,...,...,...,...,...,...,...,...
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
#df_obitos = read_data(df_obitos, 'obitos')

In [160]:
df_obitos

,"DIVULGAÇAO,MUNICÍPIO"
0,"19/03/2020,Miguel Pereira"
1,"19/03/2020,Niterói"
2,"21/03/2020,Petrópolis"
3,"23/03/2020,Rio de Janeiro"
4,"24/03/2020,Rio de Janeiro"
...,...
485,"22/04/2020,Duque de Caxias"
486,"22/04/2020,Rio de Janeiro"
487,"22/04/2020,Rio de Janeiro"
488,"22/04/2020,Rio de Janeiro"


In [217]:
# sero, queimados, japeri, mesquita, nilopolis, sao joao do meriti
metropolitana_1_obitos = ['Rio de Janeiro', 'Itaguaí', 'Serópedica', 'Queimados', 'Japeri', 'Nova Iguaçu', 'Mesquita', 'Nilopolis', 'Belford Roxo', 'Sao Joao de Meriti', 'Duque de Caxias', 'Magé']

In [218]:
df_obitos[df_obitos['MUNICÍPIO'] == metropolitana_1_obitos[9]]

,DIVULGAÇAO,MUNICÍPIO
45,03/04/2020,Sao Joao de Meriti
112,09/04/2020,Sao Joao de Meriti
186,14/04/2020,Sao Joao de Meriti
234,15/04/2020,Sao Joao de Meriti
397,19/04/2020,Sao Joao de Meriti
399,19/04/2020,Sao Joao de Meriti


In [157]:
#df_obitos.drop(axis = 1, index = 2, inplace = True)

In [207]:
df_obitos

,DIVULGAÇAO,MUNICÍPIO
0,19/03/2020,Miguel Pereira
1,19/03/2020,Niterói
2,21/03/2020,Petrópolis
3,23/03/2020,Rio de Janeiro
4,24/03/2020,Rio de Janeiro
...,...,...
485,22/04/2020,Duque de Caxias
486,22/04/2020,Rio de Janeiro
487,22/04/2020,Rio de Janeiro
488,22/04/2020,Rio de Janeiro


In [219]:
lst_ob = []
for c in metropolitana_1_obitos:
    lst_ob.append(get_data(c, df_obitos, T_fim = 24)[1])

No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados
No total 0 dados foram inutilizados


In [220]:
lst_ob

[[4,
  0,
  5,
  2,
  11,
  5,
  2,
  4,
  5,
  12,
  14,
  9,
  9,
  6,
  8,
  9,
  25,
  28,
  28,
  24,
  18,
  8,
  12,
  26],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 3, 3, 1, 0, 1, 0, 2, 2],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 3, 0, 7, 1, 0, 0, 4, 0, 1, 2, 11, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0]]

In [221]:
dic = {'DATA': dias, metropolitana_1[0]: lst[0], metropolitana_1[1]: lst_ob[1],metropolitana_1[2]: lst_ob[2],metropolitana_1[3]: lst_ob[3],metropolitana_1[4]: lst_ob[4],metropolitana_1[5]: lst_ob[5],metropolitana_1[6]: lst_ob[6],metropolitana_1[7]: lst_ob[7],metropolitana_1[8]: lst_ob[8],metropolitana_1[9]: lst_ob[9],metropolitana_1[10]: lst_ob[10],metropolitana_1[11]: lst_ob[11]}

In [222]:
dic

{'DATA': ['2020-03-29',
  '2020-03-30',
  '2020-03-31',
  '2020-04-01',
  '2020-04-02',
  '2020-04-03',
  '2020-04-04',
  '2020-04-05',
  '2020-04-06',
  '2020-04-07',
  '2020-04-08',
  '2020-04-09',
  '2020-04-10',
  '2020-04-11',
  '2020-04-12',
  '2020-04-13',
  '2020-04-14',
  '2020-04-15',
  '2020-04-16',
  '2020-04-17',
  '2020-04-18',
  '2020-04-19',
  '2020-04-20',
  '2020-04-21'],
 'RIO DE JANEIRO': [44,
  144,
  75,
  50,
  104,
  147,
  61,
  22,
  192,
  203,
  104,
  79,
  233,
  177,
  242,
  64,
  331,
  123,
  174,
  41,
  26,
  47,
  175,
  9],
 'ITAGUAI': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 'SEROPÉDICA': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'QUEIMADOS': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  2],
 'JAPERI': [0,
  0,
  0,
  0,
 

In [223]:
df_obitos_pronto = pd.DataFrame(dic)

In [224]:
df_obitos_pronto

,DATA,RIO DE JANEIRO,ITAGUAI,SEROPÉDICA,QUEIMADOS,JAPERI,NOVA IGUAÇU,MESQUITA,NILOPOLIS,BELFORD ROXO,SAO JOAO DO MERITI,DUQUE DE CAXIAS,MAGE
0,2020-03-29,44,0,0,0,0,0,0,0,0,0,0,0
1,2020-03-30,144,0,0,0,0,0,0,0,0,0,0,0
2,2020-03-31,75,0,0,0,0,0,0,0,0,0,0,0
3,2020-04-01,50,0,0,0,0,0,0,0,0,0,0,0
4,2020-04-02,104,0,0,0,0,0,0,0,1,0,0,0
5,2020-04-03,147,0,0,0,0,0,0,0,0,1,0,0
6,2020-04-04,61,0,0,0,0,2,1,0,1,0,3,0
7,2020-04-05,22,0,0,0,0,0,0,0,0,0,0,0
8,2020-04-06,192,0,0,0,0,0,0,0,1,0,0,0
9,2020-04-07,203,0,0,0,0,0,0,0,0,0,2,0


In [225]:
df_obitos_pronto.to_csv('metropolitana_1_dados_estado_obitos.csv')

In [116]:
arquivo_conf_prf = 'data_municipios/Dados_indiv_MRJ_covid19.csv'
#arquivo_obito = 'data_municipios/dados_estado_rio/αbitosASCOM.csv'
df_confirmados_prf = pd.read_csv(arquivo_conf, encoding = 'ISO-8859-1', delimiter = ',')
#df_obitos = pd.read_csv(arquivo_obito, encoding = 'ISO-8859-1', delimiter = ';')
#df_confirmados = read_data(df_confirmados, 'confirmados')
#df_obitos = read_data(df_obitos, 'obitos')

In [118]:
df_confirmados_prf = df_confirmados_prf[df_confirmados_prf['classificacao_final'].str.contains('CONFI')]

In [119]:
df_confirmados_prf

,classificacao_final,dt_notific,dt_inicio_sintomas,bairro_resid__estadia,ap_residencia_estadia,sexo,faixa_etaria,evolucao
0,CONFIRMADO,10/03/2020,08/03/2020,FORA DO MUNICï¿½PIO,N/D,M,De 30 a 39,recuperado
1,CONFIRMADO,20/03/2020,14/03/2020,COPACABANA,2.1,F,N/D,recuperado
2,CONFIRMADO,15/03/2020,12/03/2020,COPACABANA,2.1,F,De 50 a 59,recuperado
3,CONFIRMADO,24/03/2020,NaN,SAO CONRADO,2.1,M,N/D,recuperado
4,CONFIRMADO,20/03/2020,NaN,JARDIM BOTANICO,2.1,M,De 60 a 69,recuperado
...,...,...,...,...,...,...,...,...
7171,CONFIRMADO,43934,43926,ESTACIO,1,F,De 80 a 89,ï¿½bito
7172,CONFIRMADO,43936,43922,FLAMENGO,2.1,M,De 70 a 79,recuperado
7173,CONFIRMADO,43924,NaN,VAZ LOBO,3.3,M,De 80 a 89,ï¿½bito
7174,CONFIRMADO,43929,43922,ITANHANGA,4,M,De 60 a 69,ï¿½bito


In [172]:
def read_data(df, tipo):
    if tipo == 'confirmados':
        df  = df.drop(columns = ['classificacao_final', 'dt_notific', 'bairro_resid__estadia', 'ap_residencia_estadia', 'faixa_etaria', 'sexo'])
        #df.rename(mapper = {'FX_ETARIA' : 'Faixa Etaria', 'MUNIC_RESIDENCIA': 'Municipio', 'BAIRRO_RESIDENCIA': 'Bairro',
                               #'DT_NOT': 'Data'}, axis = 1, inplace = True)
        c = df.columns
        #df[[c[2], c[3]]] = df[[c[3], c[2]]]
        df.rename(mapper = {'dt_coleta / dt_notif':'Data'}, axis = 1, inplace = True)
        df = df.drop(labels = len(df)-1, axis = 0)
        return df
    elif tipo == 'obitos':
        df = df.drop(columns = ['SEXO', 'IDADE'])
        df.rename(mapper = {'FXETARIA':'Faixa Etaria', 'MUN_RES': 'Municipio', 'DT_OBITO':'Data'}, axis = 1, inplace = True)
        return df
    else:
        print('tipos são confirmados ou obitos')


def get_data(cidade, df, T_fim, T_start = '29-03-2020'):
    df_cidade = df.loc[df['MUNICÍPIO'] == cidade]
    n = len(df_cidade)
    df = df_cidade
    k = n - len(df_cidade.dropna(subset=['DIVULGAÇAO']))
    df_cidade = df_cidade.dropna(subset=['DIVULGAÇAO'])
    print('No total ' + str(k) + ' dados foram inutilizados')
    l = list(df_cidade['DIVULGAÇAO'])
    start = dt.datetime.strptime(T_start, "%d-%m-%Y")
    then = start + dt.timedelta(days=T_fim)
    days = mdates.drange(start,then,dt.timedelta(days=1))
    dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
    dados_por_dia = len(dias)*[0]
    df_cidade_new = pd.to_datetime(df_cidade['DIVULGAÇAO'], dayfirst=True, format='%d/%m/%Y')
    times =list(df_cidade_new)
    times = [str(time).split(' ')[0] for time in times]
    for i, dia in enumerate(dias):
        dados_por_dia[i] += times.count(dia)
    dados = [sum(dados_por_dia[:i]) for i in range(1, len(dados_por_dia)+1)]
    acumulado_inicial = n - dados[-1]
    dados = [acumulado_inicial + dado for dado in dados]
    return dados, dados_por_dia

In [121]:
df_conf_prf = read_data(df_confirmados_prf, 'confirmados')

In [122]:
df_conf_prf

,dt_inicio_sintomas,evolucao
0,08/03/2020,recuperado
1,14/03/2020,recuperado
2,12/03/2020,recuperado
3,NaN,recuperado
4,NaN,recuperado
...,...,...
7170,43919,ï¿½bito
7171,43926,ï¿½bito
7172,43922,recuperado
7174,43922,ï¿½bito
